In [35]:
import pysam
import numpy as np
from tqdm import tqdm

samfile1 = pysam.AlignmentFile("/Volumes/Extreme SSD/HP/cap1_FTO_test.bam", "rb")
samfile2 = pysam.AlignmentFile("/Volumes/Extreme SSD/HP/cap2_FTO_test.bam", "rb")

In [ ]:
reads1 = []
reads2 = []

for i in range(24):
    reads1.append([])
    reads2.append([])
    
i = 1
for i in range(1,25):
    if i==23:
        for read in samfile2.fetch('chrX'):
            reads2[i-1].append(read) 
        for read in samfile1.fetch('chrX'):
            reads1[i-1].append(read) 
    if i==24:
        for read in samfile2.fetch('chrY'):
            reads2[i-1].append(read) 
        for read in samfile1.fetch('chrY'):
            reads1[i-1].append(read) 
    elif i!=23 and i!=24:
        for read in samfile1.fetch(f'chr{i}'):
            reads1[i-1].append(read)
        for read in samfile2.fetch(f'chr{i}'):
            reads2[i-1].append(read)
    i+=1

In [17]:
from pyfaidx import Fasta
file_path = "/Volumes/Extreme SSD/HP/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna"
fasta = Fasta(file_path)

In [18]:
#return the base at specified chromosome and position 
def chr_loc(chromosome, position):
    try:
        basepair = fasta[chromosome][position - 1]  # pyfaidx uses 0-based indexing internally
        return basepair
    except KeyError:
        return f"Chromosome {chromosome} not found in the FASTA file."
    except IndexError:
        return f"Position {position} is out of range for chromosome {chromosome}."


In [20]:
#Removes reads in which base pair mismatches occur at the first three positions for cap2 and the first two positions for cap1
pos_count1 = []
pos_count1_m6Am = []
pos_count1_Am = []
pos_count2 = []
pos_count2_m6Am = []
pos_count2_Am = []
k = 0
for i in tqdm(range(24)):
    for j in range(len(reads1[i])):
        if reads1[i][j].is_forward==True:
            if reads1[i][j].query_sequence[0] == chr_loc(f'chr{i+1}', reads1[i][j].reference_end-39) or (reads1[i][j].query_sequence[0] == 'G' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end-39) == 'A'):
                if reads1[i][j].query_sequence[1] == chr_loc(f'chr{i+1}', reads1[i][j].reference_end-38) or (reads1[i][j].query_sequence[1] == 'G' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end-38) == 'A'): 
                    k+=1
                    if reads1[i][j].query_sequence[0] == 'A':
                        if i == 22:
                            pos_count1_m6Am.append("chrX")
                        if i == 23:
                            pos_count1_m6Am.append("chrY")
                        else:
                            pos_count1_m6Am.append(f'chr{i+1}')
                        pos_count1_m6Am.append(reads1[i][j].reference_end-40)
                        pos_count1_m6Am.append(reads1[i][j].reference_end-39)
                        pos_count1_m6Am.append("+")
                        pos_count1_m6Am.append(str(reads1[i][j].query_name).split("_")[-1] + reads1[i][j].query_sequence)   
                        pos_count1_m6Am.append(str(reads1[i][j].query_sequence))
                    elif reads1[i][j].query_sequence[0] == 'G' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end-39) == 'A':
                        if i == 22:
                            pos_count1_Am.append("chrX")
                        if i == 23:
                            pos_count1_m6Am.append("chrY")
                        else:
                            pos_count1_Am.append(f'chr{i+1}')
                        pos_count1_Am.append(reads1[i][j].reference_end-40)
                        pos_count1_Am.append(reads1[i][j].reference_end-39)
                        pos_count1_Am.append("+")
                        pos_count1_Am.append(str(reads1[i][j].query_name).split("_")[-1] + reads1[i][j].query_sequence)
                        pos_count1_Am.append(str(reads1[i][j].query_sequence))
                    else:
                        if i == 22:
                            pos_count1.append("chrX")
                        if i == 23:
                            pos_count1_m6Am.append("chrY")
                        else:
                            pos_count1.append(f'chr{i+1}')
                        pos_count1.append(reads1[i][j].reference_end-40)
                        pos_count1.append(reads1[i][j].reference_end-39)
                        pos_count1.append("+")
                        pos_count1.append(str(reads1[i][j].query_name).split("_")[-1] + reads1[i][j].query_sequence) 
                        pos_count1.append(str(reads1[i][j].query_sequence))
                    
        if reads1[i][j].is_forward==False:
            if reads1[i][j].query_sequence[39] == chr_loc(f'chr{i+1}', reads1[i][j].reference_end) or (reads1[i][j].query_sequence[39] == 'C' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end) == 'T'):
                if reads1[i][j].query_sequence[38] == chr_loc(f'chr{i+1}', reads1[i][j].reference_end-1) or (reads1[i][j].query_sequence[38] == 'C' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end-1) == 'T'): 
                    k+=1
                    if reads1[i][j].query_sequence[0] == 'T':
                        if i == 22:
                            pos_count1_m6Am.append("chrX")
                        if i == 23:
                            pos_count1_m6Am.append("chrY")
                        else:
                            pos_count1_m6Am.append(f'chr{i+1}')
                        pos_count1_m6Am.append(reads1[i][j].reference_end-1)
                        pos_count1_m6Am.append(reads1[i][j].reference_end)
                        pos_count1_m6Am.append("-")
                        pos_count1_m6Am.append(str(reads1[i][j].query_name).split("_")[-1] + reads1[i][j].query_sequence)   
                        pos_count1_m6Am.append(str(reads1[i][j].query_sequence))                    
                    elif reads1[i][j].query_sequence[39] == 'C' and chr_loc(f'chr{i+1}', reads1[i][j].reference_end-39) == 'T':
                        if i == 22:
                            pos_count1_Am.append("chrX")
                        if i == 23:
                            pos_count1_m6Am.append("chrY")
                        else:
                            pos_count1_Am.append(f'chr{i+1}')
                        pos_count1_Am.append(reads1[i][j].reference_end-1)
                        pos_count1_Am.append(reads1[i][j].reference_end)
                        pos_count1_Am.append("-")
                        pos_count1_Am.append(str(reads1[i][j].query_name).split("_")[-1] + reads1[i][j].query_sequence)
                        pos_count1_Am.append(str(reads1[i][j].query_sequence))   
                    else:
                        if i == 22:
                            pos_count1.append("chrX")
                        if i == 23:
                            pos_count1_m6Am.append("chrY")
                        else:
                            pos_count1.append(f'chr{i+1}')
                        pos_count1.append(reads1[i][j].reference_end-1)
                        pos_count1.append(reads1[i][j].reference_end)
                        pos_count1.append("-")
                        pos_count1.append(str(reads1[i][j].query_name).split("_")[-1] + reads1[i][j].query_sequence)
                        pos_count1.append(str(reads1[i][j].query_sequence))   


    for j in range(len(reads2[i])):   
        if reads2[i][j].is_forward==True:
            if reads2[i][j].query_sequence[0] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-39) or (reads2[i][j].query_sequence[0] == 'G' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-39) == 'A'):
                if reads2[i][j].query_sequence[1] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-38) or (reads2[i][j].query_sequence[1] == 'G' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-38) == 'A'):
                    if reads2[i][j].query_sequence[2] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-37) or (reads2[i][j].query_sequence[2] == 'G' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-37) == 'A'):
                        if reads2[i][j].query_sequence[0] == 'A':
                            if i == 22:
                                pos_count2_m6Am.append("chrX")
                            if i == 23:
                                pos_count1_m6Am.append("chrY")
                            else:
                                pos_count2_m6Am.append(f'chr{i+1}')
                            pos_count2_m6Am.append(reads2[i][j].reference_end-40)
                            pos_count2_m6Am.append(reads2[i][j].reference_end-39)
                            pos_count2_m6Am.append("+")
                            pos_count2_m6Am.append(str(reads2[i][j].query_name).split("_")[-1] + reads2[i][j].query_sequence)
                            pos_count2_m6Am.append(str(reads2[i][j].query_sequence))
                        elif reads2[i][j].query_sequence[0] == 'G' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-39) == 'A':
                            if i == 22:
                                pos_count2_Am.append("chrX")
                            if i == 23:
                                pos_count1_m6Am.append("chrY")
                            else:
                                pos_count2_Am.append(f'chr{i+1}')
                            pos_count2_Am.append(reads2[i][j].reference_end-40)
                            pos_count2_Am.append(reads2[i][j].reference_end-39)
                            pos_count2_Am.append("+")
                            pos_count2_Am.append(str(reads2[i][j].query_name).split("_")[-1] + reads2[i][j].query_sequence)  
                            pos_count2_Am.append(str(reads2[i][j].query_sequence))
                        else:
                            if i == 22:
                                pos_count2.append("chrX")
                            if i == 23:
                                pos_count1_m6Am.append("chrY")
                            else:
                                pos_count2.append(f'chr{i+1}')
                            pos_count2.append(reads2[i][j].reference_end-40)
                            pos_count2.append(reads2[i][j].reference_end-39)
                            pos_count2.append("+")
                            pos_count2.append(str(reads2[i][j].query_name).split("_")[-1] + reads2[i][j].query_sequence)  
                            pos_count2.append(str(reads2[i][j].query_sequence))
        if reads2[i][j].is_forward==False:
            if reads2[i][j].query_sequence[39] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end) or (reads2[i][j].query_sequence[39] == 'C' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end) == 'T'):
                if reads2[i][j].query_sequence[38] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-1) or (reads2[i][j].query_sequence[38] == 'C' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-1) == 'T'):
                    if reads2[i][j].query_sequence[37] == chr_loc(f'chr{i+1}', reads2[i][j].reference_end-2) or (reads2[i][j].query_sequence[37] == 'C' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-2) == 'T'):
                        if reads2[i][j].query_sequence[0] == 'T':
                            if i == 22:
                                pos_count2_m6Am.append("chrX")
                            if i == 23:
                                pos_count1_m6Am.append("chrY")
                            else:
                                pos_count2_m6Am.append(f'chr{i+1}')
                            pos_count2_m6Am.append(reads2[i][j].reference_end-1)
                            pos_count2_m6Am.append(reads2[i][j].reference_end)
                            pos_count2_m6Am.append("-")
                            pos_count2_m6Am.append(str(reads2[i][j].query_name).split("_")[-1] + reads2[i][j].query_sequence)   
                            pos_count2_m6Am.append(str(reads2[i][j].query_sequence))

                        elif reads2[i][j].query_sequence[39] == 'C' and chr_loc(f'chr{i+1}', reads2[i][j].reference_end-39) == 'T':
                            if i == 22:
                                pos_count2_Am.append("chrX")
                            if i == 23:
                                pos_count1_m6Am.append("chrY")
                            else:
                                pos_count2_Am.append(f'chr{i+1}')
                            pos_count2_Am.append(reads2[i][j].reference_end-1)
                            pos_count2_Am.append(reads2[i][j].reference_end)
                            pos_count2_Am.append("-")
                            pos_count2_Am.append(str(reads2[i][j].query_name).split("_")[-1] + reads2[i][j].query_sequence)
                            pos_count2_Am.append(str(reads2[i][j].query_sequence))
                        else: 
                            if i == 22:
                                pos_count2.append("chrX")
                            if i == 23:
                                pos_count1_m6Am.append("chrY")
                            else:
                                pos_count2.append(f'chr{i+1}')
                            pos_count2.append(reads2[i][j].reference_end-1)
                            pos_count2.append(reads2[i][j].reference_end)
                            pos_count2.append("-")
                            pos_count2.append(str(reads2[i][j].query_name).split("_")[-1] + reads2[i][j].query_sequence)
                            pos_count2.append(str(reads2[i][j].query_sequence))

100%|███████████████████████████████████████████| 23/23 [00:03<00:00,  6.11it/s]

1081944
279342
102846
244022
['chr1', 629188, 629189, '-', '1/1CAGCCGAACACACAAGCACCACCACAACAAACACCCCCAC', 'CAGCCGAACACACAAGCACCACCACAACAAACACCCCCAC', 'chr1', 629218, 629219, '+']


In [29]:
cap1Am = np.array(pos_count1_Am, dtype = 'object')
cap1m6Am = np.array(pos_count1_m6Am, dtype = 'object')

cap2Am = np.array(pos_count2_Am, dtype = 'object')
cap2m6Am = np.array(pos_count2_m6Am, dtype = 'object')
      
cap1Am = cap1Am.reshape((int(cap1Am.size/6), 6))
cap2Am = cap2Am.reshape((int(cap2Am.size/6), 6))

cap1m6Am = cap1m6Am.reshape((int(cap1m6Am.size/6), 6))
cap2m6Am = cap2m6Am.reshape((int(cap2m6Am.size/6), 6))

np.save("cap1_Am_nm.npy", cap1Am)
np.save("cap2_Am_nm.npy", cap2Am)
np.save("cap1_m6Am_nm.npy", cap1m6Am)
np.save("cap2_m6Am_nm.npy", cap2m6Am)